# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
#Импортируем библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
#Загружаем данные в переменную df
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
#Знакомимся с общей информацией о структуре данных
df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
#Проверим явные дубликаты
df.duplicated().sum()

153

In [5]:
#Удалим дубликаты
df = df.drop_duplicates()

#Проверяем
df.duplicated().sum()

0

In [6]:
#Приведем значения столбцов Возраст и Зарплата к целым числам
df['Возраст'] = df['Возраст'].astype('int')
df['Зарплата'] = df['Зарплата'].astype('int')

С загрузкой и подготовкой данных закончили, переходим к следующему шагу.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

**Ответ:** Не изменится.

**Обоснование:** Представим новую матрицу **Y** в качестве произведения исходной матрицы **X** и обратимой матрицы **P**
$$
Y = XP
$$

Подставим значение **Y** в изначальную формулу обучения
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

Получаем следующее выражение:
$$
w' = (P^T(X^TX)P)^{-1}X^TP^Ty = (P^T)^{-1}(X^TX)^{-1}P^{-1}X^TP^Ty = P^{-1}(X^TX)^{-1}X^Ty
$$

Мы видим, что часть полученного выражения совпадает с формулой обучения, поэтому можем записать полученное выражение так:
$$
w' = P^{-1}(X^TX)^{-1}X^Ty = P^{-1}w
$$

Теперь подставим полученное выражение в формулу предсказания:
$$
a' = Yw' = XPP^{-1}w = Xw
$$

Таким образом, **a' = a**, что говорит о неизменности предсказаний, это и требовалось доказать.

## Алгоритм преобразования

**Алгоритм**

1. Генерируем случайную матрицу
2. Проверяем ее на обратимость (если обратима - идем дальше, если нет - генерируем другую матрицу)
3. Умножаем матрицу признаков на сгенерированную матрицу и получаем итоговую матрицу с зашифрованными данными клиентов

**Обоснование**

Алгоритм даст возможность зашифровать данные клиента без какого-либо влияния на качество работы модели.

## Проверка алгоритма

In [7]:
#Задаем переменные для признаков и целевого показателя
features = df.drop(['Страховые выплаты'], axis = 1)
target = df['Страховые выплаты']

In [8]:
#Разделим выборку на обучающую и тестовую
train_features, test_features, train_target, test_target = train_test_split(
    features, target, test_size=0.25, random_state=12345)

#Проверяем
print(train_features.shape)
print(test_features.shape)
print(train_target.shape)
print(test_target.shape)

(3635, 4)
(1212, 4)
(3635,)
(1212,)


In [9]:
#Строим модель линейной регрессии
model = LinearRegression()

model.fit(train_features, train_target)
predictions = model.predict(test_features)

r2 = r2_score(test_target, predictions)

print(f'Значение метрики R2 при линейной регрессии без преобразования признаков:{r2:.3}')

Значение метрики R2 при линейной регрессии без преобразования признаков:0.423


Теперь проверим значение линейной регрессии для преобразованных данных. Для этого пройдемся по нашему алгоритму, после чего измерим метрику R2.

In [10]:
#Генерируем случайную матрицу и проверяем на обратимость
shape = train_features.shape[1]
random_matrix = np.random.rand(shape, shape)

det = np.linalg.det(random_matrix)
while det == 0:
    random_matrix = np.random.rand(shape, shape)
    det = np.linalg.det(random_matrix)
    
inverse_matrix = np.linalg.inv(random_matrix)

In [11]:
#Преобразуем признаки
train_features_transformed = np.dot(train_features, inverse_matrix)
test_features_transformed = np.dot(test_features, inverse_matrix)

In [12]:
#Строим модель линейной регрессии с преобразованными признаками
model.fit(train_features_transformed, train_target)
predictions_transformed = model.predict(test_features_transformed)

r2_transformed = r2_score(test_target, predictions_transformed)

print(f'Значение метрики R2 при линейной регрессии с преобразованием признаков:{r2_transformed:.3}')

Значение метрики R2 при линейной регрессии с преобразованием признаков:0.423


In [13]:
#Сравним значения R2 с преобразованием признаков и без

r2 - r2_transformed

1.0340617251358708e-12

Как видим, разница между полученными значениями метрик R2 ничтожно мала, а значит можно считать, что они равны.

### Итоги:
1. Мы загрузили и подготовили данные для работы
2. Доказали, что качество линейной регрессии не меняется при умножении признаком на обратимую матрицу
3. Описали алгоритм преобразования
4. Применили алгоритм преобразования на практике и увидели, что значения метрики R2 с преобразованием и без преобразования идентичны.

Таким образом, мы смогли защитить данные клиентов, не повлияв на качество модели.